In [1]:
!pip install -q langchain --user --upgrade --force-reinstall --progress-bar off

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opensearch-py 2.0.1 requires urllib3<2,>=1.21.1, but you have urllib3 2.2.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
jupyter-server 2.1.0 requires anyio<4,>=3.1.0, but you have anyio 4.2.0 which is incompatible.
gremlinpython 3.6.1 requires aiohttp<=3.8.1,>=3.8.0, but you have aiohttp 3.9

In [2]:
!pip install -q openai --progress-bar off

In [1]:
temp = 0.0
num_labels = 17
model_name="gpt-3.5-turbo-0301"
# model_name="gpt-3.5-turbo-instruct"
# model_name="gpt-4-0613"

In [2]:
output_dir = "../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/"

final_xlsx_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}.xlsx"
print(final_xlsx_filename)

classification_report_filename = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_classification_report.xlsx"
print(classification_report_filename)

final_xlsx_filename_corrected = output_dir+f"gpt_1000_texts_{num_labels}sdg_temperature_{temp}_model_name_{model_name}_corrected.xlsx"
print(final_xlsx_filename_corrected)

../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx
../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx
../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_corrected.xlsx


In [5]:
import os
from IPython.display import clear_output
os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
clear_output()

In [7]:
import pandas as pd
import pickle
from sklearn.metrics import classification_report

In [6]:
import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

from typing import Union
from IPython.display import display, HTML

In [7]:
def perform_classification(text: str, chain: LLMChain) -> str:
    """Perform classification using LangChain model API.

    Parameters
    ----------
    text : str
        Input text.
    chain : LLMChain
        LangChain model.

    Returns
    -------
    str or None
        API Response
    """
    try:
        return chain.run(text=text)
    except Exception as e:
        print(e)
        return None
    
def convert_to_dict(row: str) -> dict:
    try:
        row = row.replace("false", "False")
        row = row.replace("true", "True")
        return ast.literal_eval(row)
    except Exception as e:
        print(e, row)
        return row

In [8]:
llm = ChatOpenAI(temperature=temp, model_name=model_name, max_tokens=500)

/home/jovyan/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Caricamento dati e preparazione prompt


In [9]:
# IN QUESTO ESPERIMENTO CONSIDERO LO STESSO DATASET UTILIZZATO IN 
# GPT_zero-shot_learning/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.ipynb
# IN MODO CHE I DATI SIANO GLI STESSI 

# data = pd.read_excel("../GPT_zero-shot_learning/output/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# # data.index = data.text_id
# # data.text_id = data.index
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
# data

# data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")

data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124
...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109


In [19]:
# data.shape

In [20]:
# data.groupby('sdg').count()['text']

In [12]:
#data_sel_4_few_shot_examples = pd.read_excel("../GPT_few-shot_learning/data/output/2023.10.19-1_GPT_few-shot-learning_all-17-sdg/data_sel_4_few_shot_examples.xlsx")

In [13]:
#data_sel_4_few_shot_examples

In [14]:
# https://www.un.org/en/exhibits/page/sdgs-17-goals-transform-world#:~:text=GOAL%201%3A%20No%20Poverty%20GOAL,10%3A%20Reduced%20Inequalities%20GOAL%2011

template = """
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes:

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The text to be processed is:

```
{text}
```
"""

In [15]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [16]:
chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
output_datapath = output_dir

In [35]:
# data_sel = data_sel_4_few_shot_examples
# data_sel = data
# data_sel = data[data.sdg==0].sample(10)
data_sel = data #.sample(3)

In [36]:
# results = data_sel.apply(lambda x: perform_classication(x["text"], chain), axis=1)
import time

results = []
for index, row in data_sel.iterrows():
    cnt = index
    result = perform_classification(row["text"], chain)
    results.append((row["text_id"], result))
    
    result_extended = (cnt, row["text_id"], row["sdg"], result)

    print(result_extended)

    with open(output_datapath+f'/sdg_classification_GPT_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)
    
    time.sleep(5)

(0, 6668, 0, 'SDG 9: Industry, Innovation and Infrastructure.')
(1, 6099, 0, 'SDG 0')
(2, 6471, 0, 'SDG 16: Peace, Justice and Strong Institutions. The text discusses the suspension of a life sentence and the granting of conditional bail to a convict in a high-profile arms haul case. The judges also make observations about the lack of evidence to support certain claims made by co-accused in their confessionals. This relates to the goal of promoting peaceful and inclusive societies for sustainable development, providing access to justice for all, and building effective, accountable and inclusive institutions at all levels.')
(3, 1786, 0, 'SDG 9: Industry, Innovation and Infrastructure.')
(4, 2143, 0, "SDG 7: Affordable and Clean Energy. The text discusses the pricing of gas generated for electricity in Nigeria and the impact it has on the country's economy and citizens. This relates to SDG 7, which aims to ensure access to affordable, reliable, sustainable, and modern energy for all.")


In [37]:
results

[(6668, 'SDG 9: Industry, Innovation and Infrastructure.'),
 (6099, 'SDG 0'),
 (6471,
  'SDG 16: Peace, Justice and Strong Institutions. The text discusses the suspension of a life sentence and the granting of conditional bail to a convict in a high-profile arms haul case. The judges also make observations about the lack of evidence to support certain claims made by co-accused in their confessionals. This relates to the goal of promoting peaceful and inclusive societies for sustainable development, providing access to justice for all, and building effective, accountable and inclusive institutions at all levels.'),
 (1786, 'SDG 9: Industry, Innovation and Infrastructure.'),
 (2143,
  "SDG 7: Affordable and Clean Energy. The text discusses the pricing of gas generated for electricity in Nigeria and the impact it has on the country's economy and citizens. This relates to SDG 7, which aims to ensure access to affordable, reliable, sustainable, and modern energy for all."),
 (3377, 'SDG 0')

In [38]:
# results_processed = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["motivation_for_the_answer", "detected_goals", "selected_goal"]]
# results_processed

In [39]:
# results_processed2 = results.apply(convert_to_dict).apply(pd.Series).loc[:, ["goal_label", "motivation_for_the_answer", "sdg_probability"]]
# results_processed2

In [40]:
results_processed = pd.DataFrame(results, columns=['text_id', 'result'])
results_processed['label_pred'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,text_id,result,label_pred
0,6668,"SDG 9: Industry, Innovation and Infrastructure.",9
1,6099,SDG 0,0
2,6471,"SDG 16: Peace, Justice and Strong Institutions...",16
3,1786,"SDG 9: Industry, Innovation and Infrastructure.",9
4,2143,SDG 7: Affordable and Clean Energy. The text d...,7
...,...,...,...
1015,4695,"SDG 16: Peace, Justice and Strong Institutions...",16
1016,6628,"SDG 16: Peace, Justice and Strong Institutions...",16
1017,4563,"SDG 16: Peace, Justice and Strong Institutions...",16
1018,6710,"SDG 16: Peace, Justice and Strong Institutions.",16


In [41]:
# final = data_sel.merge(results_processed, left_index=True, right_index=True)
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
# final = data_sel.merge(results_processed2, left_index=True, right_index=True)
# final = final.drop(columns='Unnamed: 0')
final

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,result,label_pred
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,"SDG 9: Industry, Innovation and Infrastructure.",9
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,SDG 0,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,"SDG 16: Peace, Justice and Strong Institutions...",16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,"SDG 9: Industry, Innovation and Infrastructure.",9
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,SDG 7: Affordable and Clean Energy. The text d...,7
...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,"SDG 16: Peace, Justice and Strong Institutions...",16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,"SDG 16: Peace, Justice and Strong Institutions...",16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,"SDG 16: Peace, Justice and Strong Institutions...",16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,"SDG 16: Peace, Justice and Strong Institutions.",16


In [42]:
final_xlsx_filename

'../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301.xlsx'

In [43]:
final.to_excel(final_xlsx_filename)

In [4]:
final = pd.read_excel(final_xlsx_filename)

In [5]:
final_corrected = final

In [8]:
import numpy as np
y_true = final_corrected.sdg 
y_pred = final_corrected.label_pred
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.30      0.46        60
           1       0.68      0.83      0.75        60
           2       0.95      0.88      0.91        60
           3       0.69      0.97      0.81        60
           4       0.88      0.93      0.90        60
           5       0.92      0.90      0.91        60
           6       0.98      0.93      0.96        60
           7       0.87      0.87      0.87        60
           8       0.53      0.75      0.62        60
           9       0.70      0.77      0.73        60
          10       0.82      0.47      0.60        60
          11       0.89      0.80      0.84        60
          12       0.90      0.88      0.89        60
          13       0.79      0.88      0.83        60
          14       0.92      1.00      0.96        60
          15       0.91      0.87      0.89        60
          16       0.91      0.87      0.89        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
results_processed_merged = final
results_processed_merged.head(3)

,Unnamed: 0,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,result,label_pred
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,"SDG 9: Industry, Innovation and Infrastructure.",9
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,SDG 0,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,"SDG 16: Peace, Justice and Strong Institutions...",16


In [13]:
results_processed_merged = results_processed_merged.rename(columns={'label_pred': 'predicted_label_clean'})

In [14]:
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if x>16 else x)

In [16]:
import numpy as np
y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.78      0.30      0.43        60
           1       0.68      0.83      0.75        60
           2       0.95      0.88      0.91        60
           3       0.69      0.97      0.81        60
           4       0.88      0.93      0.90        60
           5       0.92      0.90      0.91        60
           6       0.98      0.93      0.96        60
           7       0.87      0.87      0.87        60
           8       0.53      0.75      0.62        60
           9       0.70      0.77      0.73        60
          10       0.82      0.47      0.60        60
          11       0.89      0.80      0.84        60
          12       0.90      0.88      0.89        60
          13       0.79      0.88      0.83        60
          14       0.92      1.00      0.96        60
          15       0.91      0.87      0.89        60
          16       0.91      0.87      0.89        60

    accuracy    

In [17]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)


In [18]:
classification_report_filename

'../GPT_zero-shot_learning/data/output/2024.02.14-01_GPT-3.5_zero-shot-learning_RIESECUZIONE_PER_VERIFICA_FINALE_PAPER/gpt_1000_texts_17sdg_temperature_0.0_model_name_gpt-3.5-turbo-0301_classification_report.xlsx'

In [19]:
report_df[['precision', 'recall', 'f1-score']] *= 100

In [20]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [21]:
print(precision, recall, accuracy, f1_score)

83.0565291675998 81.76470588235296 81.76470588235294 81.12683430902669


In [22]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

83.1 81.8 81.8 81.1
